In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import TomekLinks
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool
#Import other necessary model libraries, for this example, using Logistic Regression

In [2]:
train_data = pd.read_csv("../data/train_dataset.csv")

In [3]:
train_data['next_month_plan'] = train_data['next_month_plan'].astype('category')

In [4]:
train_data['next_month_plan'].unique()

['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']
Categories (8, object): ['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']

In [5]:
labels_x = ['PKG0','PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7', 'PKG8']

In [6]:
le = preprocessing.LabelEncoder()

In [7]:
le.fit(labels_x)

LabelEncoder()

In [8]:
train_data['encoded_class_labels'] = le.transform(train_data['next_month_plan'])

In [9]:
train_data['device_category'].unique()

array(['Smartphone', 'Basic', 'Feature phone', 'Pluggable card', 'Tablet',
       nan, 'Modem'], dtype=object)

In [10]:
train_data['device_type'] = train_data['device_type'].fillna(value = 'Unknown')
train_data['device_category'] = train_data['device_category'].fillna(value = 'Unknown')
train_data['gender'] = train_data['gender'].fillna(value = 'Unknown')
train_data['age_group'] = train_data['age_group'].fillna(value = 'Unknown')

In [11]:
train_data = train_data.drop(columns = ['next_month_plan'])

In [12]:
train_data['dusage_avg'] = train_data['dusage_avg'].fillna(value = 0)
train_data['vusage_offnet_avg'] = train_data['vusage_offnet_avg'].fillna(value = 0)
train_data['add_on_tot_rental'] = train_data['add_on_tot_rental'].fillna(value = -1)
train_data['add_on_count'] = train_data['add_on_count'].fillna(value = 0)
train_data['vusage_onnet_avg'] = train_data['vusage_onnet_avg'].fillna(value = 0)

In [13]:
def get_col_types(dataframe):
    data_cat_cols = []
    data_quan_cols = []
    
    for col in np.array(dataframe.columns):
        if dataframe[col].dtype == 'int64' or dataframe[col].dtype == 'float64':
            data_quan_cols.append(col)
        elif dataframe[col].dtype == 'O':
            data_cat_cols.append(col)
            
    return data_cat_cols, data_quan_cols

In [14]:
X = train_data.drop(columns=['encoded_class_labels', 'primary_identifier'])

In [15]:
data_cat_cols, data_quan_cols = get_col_types(X)

In [16]:
y = train_data['encoded_class_labels']

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=10, test_size = 0.05, stratify = y)

In [18]:
quan_pipeline = Pipeline([
    ('std_scaler', RobustScaler())
])

quan_transformed = quan_pipeline.fit_transform(X_train[data_quan_cols])

In [19]:
data_pipeline = ColumnTransformer([
    ('numerical', quan_pipeline, data_quan_cols),
    ('categorical', OrdinalEncoder(), data_cat_cols),
    
])

train_data_processed = data_pipeline.fit_transform(X_train)

In [20]:
test_data_processed = data_pipeline.transform(X_test)

In [54]:
model_cb = CatBoostClassifier(early_stopping_rounds=100, n_estimators=2000)
model_cb.fit(train_data_processed, Y_train, plot=True, eval_set=(test_data_processed, Y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.086759
0:	learn: 1.9280575	test: 1.9226005	best: 1.9226005 (0)	total: 21.8ms	remaining: 43.7s
1:	learn: 1.8190385	test: 1.8144847	best: 1.8144847 (1)	total: 37ms	remaining: 37s
2:	learn: 1.7381213	test: 1.7355724	best: 1.7355724 (2)	total: 56.3ms	remaining: 37.5s
3:	learn: 1.6778376	test: 1.6771039	best: 1.6771039 (3)	total: 76.2ms	remaining: 38s
4:	learn: 1.6254117	test: 1.6232697	best: 1.6232697 (4)	total: 91.5ms	remaining: 36.5s
5:	learn: 1.5808453	test: 1.5795967	best: 1.5795967 (5)	total: 106ms	remaining: 35.1s
6:	learn: 1.5427321	test: 1.5414343	best: 1.5414343 (6)	total: 121ms	remaining: 34.5s
7:	learn: 1.5094716	test: 1.5083477	best: 1.5083477 (7)	total: 135ms	remaining: 33.7s
8:	learn: 1.4813277	test: 1.4816599	best: 1.4816599 (8)	total: 149ms	remaining: 33s
9:	learn: 1.4575483	test: 1.4580327	best: 1.4580327 (9)	total: 163ms	remaining: 32.5s
10:	learn: 1.4338787	test: 1.4343382	best: 1.4343382 (10)	total: 176ms	remaining: 31.8s
11:	learn: 1.4133736	test

98:	learn: 1.0780047	test: 1.1080073	best: 1.1080073 (98)	total: 1.45s	remaining: 27.8s
99:	learn: 1.0768874	test: 1.1077157	best: 1.1077157 (99)	total: 1.46s	remaining: 27.8s
100:	learn: 1.0759722	test: 1.1069283	best: 1.1069283 (100)	total: 1.47s	remaining: 27.7s
101:	learn: 1.0745382	test: 1.1059507	best: 1.1059507 (101)	total: 1.49s	remaining: 27.7s
102:	learn: 1.0729878	test: 1.1051542	best: 1.1051542 (102)	total: 1.5s	remaining: 27.7s
103:	learn: 1.0721532	test: 1.1047710	best: 1.1047710 (103)	total: 1.51s	remaining: 27.6s
104:	learn: 1.0706712	test: 1.1032486	best: 1.1032486 (104)	total: 1.53s	remaining: 27.7s
105:	learn: 1.0699180	test: 1.1031629	best: 1.1031629 (105)	total: 1.55s	remaining: 27.7s
106:	learn: 1.0688744	test: 1.1028191	best: 1.1028191 (106)	total: 1.56s	remaining: 27.7s
107:	learn: 1.0678257	test: 1.1026004	best: 1.1026004 (107)	total: 1.58s	remaining: 27.6s
108:	learn: 1.0666904	test: 1.1023969	best: 1.1023969 (108)	total: 1.59s	remaining: 27.6s
109:	learn: 1.0

201:	learn: 0.9792990	test: 1.0749054	best: 1.0743393 (198)	total: 2.89s	remaining: 25.8s
202:	learn: 0.9790603	test: 1.0751222	best: 1.0743393 (198)	total: 2.9s	remaining: 25.7s
203:	learn: 0.9782700	test: 1.0745442	best: 1.0743393 (198)	total: 2.92s	remaining: 25.7s
204:	learn: 0.9772518	test: 1.0745500	best: 1.0743393 (198)	total: 2.94s	remaining: 25.7s
205:	learn: 0.9760616	test: 1.0740797	best: 1.0740797 (205)	total: 2.96s	remaining: 25.8s
206:	learn: 0.9757261	test: 1.0741103	best: 1.0740797 (205)	total: 2.97s	remaining: 25.8s
207:	learn: 0.9749103	test: 1.0740679	best: 1.0740679 (207)	total: 2.99s	remaining: 25.7s
208:	learn: 0.9745166	test: 1.0735437	best: 1.0735437 (208)	total: 3s	remaining: 25.7s
209:	learn: 0.9734917	test: 1.0736909	best: 1.0735437 (208)	total: 3.01s	remaining: 25.7s
210:	learn: 0.9728172	test: 1.0736450	best: 1.0735437 (208)	total: 3.02s	remaining: 25.6s
211:	learn: 0.9720781	test: 1.0732465	best: 1.0732465 (211)	total: 3.04s	remaining: 25.6s
212:	learn: 0.

304:	learn: 0.9101689	test: 1.0629197	best: 1.0629197 (304)	total: 4.32s	remaining: 24s
305:	learn: 0.9095757	test: 1.0626632	best: 1.0626632 (305)	total: 4.34s	remaining: 24s
306:	learn: 0.9093023	test: 1.0630674	best: 1.0626632 (305)	total: 4.35s	remaining: 24s
307:	learn: 0.9086860	test: 1.0630527	best: 1.0626632 (305)	total: 4.37s	remaining: 24s
308:	learn: 0.9080657	test: 1.0629997	best: 1.0626632 (305)	total: 4.38s	remaining: 24s
309:	learn: 0.9076130	test: 1.0632946	best: 1.0626632 (305)	total: 4.4s	remaining: 24s
310:	learn: 0.9067535	test: 1.0631934	best: 1.0626632 (305)	total: 4.41s	remaining: 24s
311:	learn: 0.9062733	test: 1.0631002	best: 1.0626632 (305)	total: 4.42s	remaining: 23.9s
312:	learn: 0.9058405	test: 1.0632182	best: 1.0626632 (305)	total: 4.44s	remaining: 23.9s
313:	learn: 0.9055267	test: 1.0629549	best: 1.0626632 (305)	total: 4.45s	remaining: 23.9s
314:	learn: 0.9048505	test: 1.0630125	best: 1.0626632 (305)	total: 4.46s	remaining: 23.9s
315:	learn: 0.9042341	tes

407:	learn: 0.8532124	test: 1.0649920	best: 1.0625266 (326)	total: 5.77s	remaining: 22.5s
408:	learn: 0.8525027	test: 1.0647824	best: 1.0625266 (326)	total: 5.78s	remaining: 22.5s
409:	learn: 0.8521684	test: 1.0645937	best: 1.0625266 (326)	total: 5.79s	remaining: 22.5s
410:	learn: 0.8519080	test: 1.0645339	best: 1.0625266 (326)	total: 5.81s	remaining: 22.5s
411:	learn: 0.8508693	test: 1.0646985	best: 1.0625266 (326)	total: 5.82s	remaining: 22.4s
412:	learn: 0.8500734	test: 1.0644949	best: 1.0625266 (326)	total: 5.84s	remaining: 22.4s
413:	learn: 0.8495668	test: 1.0645436	best: 1.0625266 (326)	total: 5.85s	remaining: 22.4s
414:	learn: 0.8491217	test: 1.0644655	best: 1.0625266 (326)	total: 5.86s	remaining: 22.4s
415:	learn: 0.8484092	test: 1.0644206	best: 1.0625266 (326)	total: 5.88s	remaining: 22.4s
416:	learn: 0.8482306	test: 1.0643375	best: 1.0625266 (326)	total: 5.89s	remaining: 22.3s
417:	learn: 0.8477146	test: 1.0647129	best: 1.0625266 (326)	total: 5.9s	remaining: 22.3s
418:	learn:

In [55]:
y_pred = model_cb.predict(test_data_processed)

In [56]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.72      0.82      0.77       183
           2       0.64      0.13      0.22        68
           3       0.44      0.72      0.55        86
           4       0.31      0.11      0.16        37
           5       0.66      0.82      0.73        78
           6       0.50      0.35      0.41        37
           7       0.50      0.17      0.25        18
           8       0.47      0.50      0.49        18

    accuracy                           0.60       525
   macro avg       0.53      0.45      0.45       525
weighted avg       0.59      0.60      0.56       525



In [ ]:
shap_values = model_cb.get_feature_importance(Pool(X, y), type='ShapValues')

In [ ]:
shap_values.